In [1]:
import gym
import gym
import gym.wrappers

import matplotlib.pyplot as plt

import multiprocessing
import neat
from neat import nn, population, statistics, Config
import numpy as np
import os
import pickle
import random
import time
from pathlib import Path

CONFIGS_PATH = Path('../config')

In [2]:
env = gym.make('CartPole-v1')

config_path = CONFIGS_PATH / 'config-cart-pole-v0'
config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    str(config_path),
)

In [3]:
def simulate_species(net, env, episodes=1, steps=5000, render=False):
    fitnesses = []
    for runs in range(episodes):
        inputs = env.reset()
        cum_reward = 0.0
        for j in range(steps):
            outputs = net.activate(inputs)
            action = np.argmax(outputs)
            inputs, reward, done, _ = env.step(action)
            if render:
                env.render()
            if done:
                break
            cum_reward += reward

        fitnesses.append(cum_reward)

    fitness = np.array(fitnesses).mean()
    return fitness

def evaluate_genome(g, config):
    net = nn.FeedForwardNetwork.create(g, config)
    return simulate_species(net, env, render=False,)

def eval_fitness(genomes, config):
    for g_id, g in genomes:
        fitness = evaluate_genome(g, config)
        g.fitness = fitness

pop = population.Population(config)
pop.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
pop.add_reporter(stats)
pop.add_reporter(neat.Checkpointer(5))

winner = pop.run(eval_fitness, 100)


 ****** Running generation 0 ****** 

Population's average fitness: 18.85500 stdev: 24.17590
Best fitness: 143.00000 - size: (2, 8) - species 1 - id 194
Average adjusted fitness: 0.087
Mean genetic distance 1.146, standard deviation 0.286
Population of 200 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   200    143.0    0.087     0
Total extinctions: 0
Generation time: 0.108 sec

 ****** Running generation 1 ****** 

Population's average fitness: 43.70500 stdev: 79.53790
Best fitness: 499.00000 - size: (3, 8) - species 1 - id 219
Average adjusted fitness: 0.075
Mean genetic distance 1.204, standard deviation 0.309
Population of 200 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   200    499.0    0.075     0
Total extinctions: 0
Generation time: 0.204 sec (0.156 average)

 ****** Running generation 2 ****** 

Population's average fitness: 54.4650

In [5]:
# env = gym.make('CartPole-v1')
from neat.checkpoint import Checkpointer

# population = Checkpointer.restore_checkpoint('neat-checkpoint-999')

# winner = stats.best_genome()
# winner = population.generation
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
    
inputs = env.reset()
steps = 2000
for _ in range(steps):
    outputs = winner_net.activate(inputs)
    action = np.argmax(outputs)
    inputs, reward, done, _ = env.step(action)
    env.render()

print("completed!")
env.close()

completed!


In [16]:
winner = population.population[170516]